# Jupyter notebook to quickly visualize stripes using Higlass

<details>
<summary>LICENCE</summary>

Copyright (C) 2024 Roberto Rossini \<roberros@uio.no\>
SPDX-License-Identifier: MIT
</details>

In [ ]:
from clodius.cli import aggregate
import hictkpy
import higlass as hg

import pathlib
from typing import Dict

In [ ]:
def _init_tmpdir():
    import tempfile

    global _tmpdir
    try:
        _tmpdir = pathlib.Path(tempfile.mkdtemp())
    except:
        del _tmpdir
        raise


def _cleanup_tmpdir():
    import shutil

    if _tmpdir is not None:
        shutil.rmtree(_tmpdir)


## Setup self-deleting tmpdir
if "_tmpdir" not in globals():
    _tmpdir = None
    import atexit

    _init_tmpdir()
    atexit.register(_cleanup_tmpdir)


def ensure_file_exists(arg: str) -> pathlib.Path:
    if (path := pathlib.Path(arg)).is_file():
        return path

    raise FileNotFoundError(arg)


def bedpe_to_beddb(
    path_to_bedpe: pathlib.Path, chrom_sizes: pathlib.Path, outdir: pathlib.Path = _tmpdir
) -> pathlib.Path:
    out_path = outdir / f"{path_to_bedpe.stem}.beddb"
    aggregate._bedpe(
        str(path_to_bedpe),
        str(out_path),
        chromsizes_filename=str(chrom_sizes),
        chr1_col=1,
        from1_col=2,
        to1_col=3,
        chr2_col=4,
        from2_col=5,
        to2_col=6,
    )
    return out_path


def fetch_chromsizes(mcool: pathlib.Path) -> Dict[str, int]:
    return hictkpy.MultiResFile(mcool).chromosomes()


def dump_chrom_sizes(chroms: Dict[str, int], outdir: pathlib.Path = _tmpdir) -> pathlib.Path:
    dest = outdir / "chrom.sizes"

    with dest.open("w") as f:
        for chrom, size in chroms.items():
            f.write(f"{chrom}\t{size}\n")

    return dest


def setup_higlass(mcool: pathlib.Path, stripes: pathlib.Path):
    stripe_track = hg.bed2ddb(str(stripes)).track("2d-rectangle-domains")
    heatmap_track = hg.combine(hg.cooler(str(mcool)).track("heatmap", height=500), stripe_track, height=500)

    width = 12
    return hg.view(
        (hg.cooler(str(mcool)).track("chromosome-labels"), "left"),
        (hg.cooler(str(mcool)).track("chromosome-labels"), "top"),
        (heatmap_track, "center"),
        width=width,
    )

## Required inputs

- `mcool`: path to the .mcool file used to call stripes
- `bedpe`: path to a BEDPE with the stripes (generated with e.g. stripepy view)

In [ ]:
mcool = ensure_file_exists("CHANGEME.mcool")
bedpe = ensure_file_exists("CHANGEME.bedpe")

## Launch Higlass

In [ ]:
chroms = fetch_chromsizes(mcool)

chrom_sizes_file = dump_chrom_sizes(chroms)
stripes_beddb = bedpe_to_beddb(bedpe, chrom_sizes_file)

view = setup_higlass(mcool, stripes_beddb)

view